In [1]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

1.14.0


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

11493376/11490434 [==============================] - 3s 0us/step


In [3]:
# 定义模型：演示如何保存和加载权重
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])

    return model


model = create_model()
model.summary()

W0808 17:46:45.559504 4501185984 deprecation.py:506] From /Users/skylar/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
# 在训练期间保存检查点， 从而可以从上次暂停的地方继续训练
# 训练模型，并将ModelCheckPoint回调传递给该模型
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model = create_model()
model.fit(train_images, train_labels, epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 544/1000 [===============>..............] - ETA: 0s - loss: 1.5742 - acc: 0.5423
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 369us/sample - loss: 1.1885 - acc: 0.6700 - val_loss: 0.7537 - val_acc: 0.7520
Epoch 2/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.4601 - acc: 0.8587
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 130us/sample - loss: 0.4418 - acc: 0.8630 - val_loss: 0.5590 - val_acc: 0.8320
Epoch 3/10
 640/1000 [==================>...........] - ETA: 0s - loss: 0.2870 - acc: 0.9219
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 194us/sample - loss: 0.2910 - acc: 0.9220 - val_loss: 0.5106 - val_acc: 0.8270
Epoch 4/10
 448/1000 [============>.................] - ETA: 0s - loss: 0.1863 - acc: 0.9598
Epoch 00004: saving model to training_1/cp.ckpt
1000/10

In [5]:
# 创建一个未经训练的全新模型，仅通过权重恢复模型。 由于模型架构相同，因此不同的模型实例可以分享权重
model = create_model()
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuray: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 61us/sample - loss: 2.3556 - acc: 0.0930
Untrained model, accuray:  9.30%


In [6]:
# 然后从检查点加载权重，并重新评估
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 49us/sample - loss: 0.4115 - acc: 0.8720
Restored model, accuracy: 87.20%


In [7]:
# ModelCheckpoint回调提供多个选项，用于为生成的检查点提供独一无二的名称，以及调整检查点创建频率
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only=True,
                                                 period=5)  # save weights, every 5-epochs

model = create_model()
model.fit(train_images, train_labels, epochs=50, callbacks=[cp_callback],
          validation_data=(test_images, test_labels), verbose=0)

W0808 18:10:02.075483 4501185984 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [10]:
# 生成的检查点
ls{checkpoint_dir}

checkpoint                        cp-0030.ckpt.data-00000-of-00002
cp-0005.ckpt.data-00000-of-00002  cp-0030.ckpt.data-00001-of-00002
cp-0005.ckpt.data-00001-of-00002  cp-0030.ckpt.index
cp-0005.ckpt.index                cp-0035.ckpt.data-00000-of-00002
cp-0010.ckpt.data-00000-of-00002  cp-0035.ckpt.data-00001-of-00002
cp-0010.ckpt.data-00001-of-00002  cp-0035.ckpt.index
cp-0010.ckpt.index                cp-0040.ckpt.data-00000-of-00002
cp-0015.ckpt.data-00000-of-00002  cp-0040.ckpt.data-00001-of-00002
cp-0015.ckpt.data-00001-of-00002  cp-0040.ckpt.index
cp-0015.ckpt.index                cp-0045.ckpt.data-00000-of-00002
cp-0020.ckpt.data-00000-of-00002  cp-0045.ckpt.data-00001-of-00002
cp-0020.ckpt.data-00001-of-00002  cp-0045.ckpt.index
cp-0020.ckpt.index                cp-0050.ckpt.data-00000-of-00002
cp-0025.ckpt.data-00000-of-00002  cp-0050.ckpt.data-00001-of-00002
cp-0025.ckpt.data-00001-of-00002  cp-0050.ckpt.index
cp-0025.ckpt.index


In [11]:
# 最新的检查点
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [12]:
# 重置模型并加载最新的检查点
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 76us/sample - loss: 0.4901 - acc: 0.8740
Restored model, accuracy: 87.40%


In [13]:
# 保存权重
model.save_weights('./checkpoints/my_checkpoint')

# 恢复权重
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 77us/sample - loss: 0.4901 - acc: 0.8740
Restored model, accuracy: 87.40%


In [14]:
# 保存整个模型：包含权重、模型配置乃至优化器配置
# Keras使用HDF5标准提供基本的保存格式。可将保存的模型视为一个二进制blob
model = create_model()
model.fit(train_images, train_labels, epochs=5)
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 152us/sample - loss: 1.1840 - acc: 0.6620
Epoch 2/5
1000/1000 [==============================] - 0s 85us/sample - loss: 0.4325 - acc: 0.8710
Epoch 3/5
1000/1000 [==============================] - 0s 89us/sample - loss: 0.2827 - acc: 0.9340
Epoch 4/5
1000/1000 [==============================] - 0s 84us/sample - loss: 0.2064 - acc: 0.9540
Epoch 5/5
1000/1000 [==============================] - 0s 78us/sample - loss: 0.1566 - acc: 0.9700


In [15]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

W0808 18:28:03.621572 4501185984 deprecation.py:506] From /Users/skylar/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0808 18:28:03.624340 4501185984 deprecation.py:506] From /Users/skylar/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0808 18:28:03.910509 4501185984 util.py:244] Unresolved object in checkpoint: (root).optimizer.iter
W0808 18:28:03.911194 4501185984 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_1
W0

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
# 检查其准确率
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 96us/sample - loss: 0.4272 - acc: 0.8560
Restored model, accuracy: 85.60%


In [ ]:
# keras通过检查架构来保存模型。目前，无法保存TensorFlow优化器（来自tf.train）。
# 使用此类优化器时，需要在加载模型后对其进行重新编译，使优化器的状态变松散。